## BRONZE TO SILVER LAYER

### Silver Layer - Stock Price History


In [1]:
# Importing Common Utility Funcation
%run ../00_COMMON/common_utility.ipynb

In [2]:
# Import necessary libraries and utility functions
import re

import pandas as pd

from ETLTools import GlobalPath

In [3]:
# Instantiate GlobalPath
stockdata_bronze_layer_path = GlobalPath("DATA/BRONZE/StockData")
stockprice_silver_file_path = GlobalPath(
    "DATA/SILVER/StockPrice/StockPrice_data.csv"
)

### Data Processing

- Initialize an empty list to store DataFrames.
- Read and concatenate data from multiple CSV files.


In [4]:
# Initialize an empty list to store individual stock price DataFrames
df_stock_price_list = []

# Generate file paths for available CSV files in the Bronze layer
file_paths = check_files_availability(
    stockdata_bronze_layer_path, file_pattern="*.csv"
)

# Loop through the list of CSV files in the folder
for file_path in file_paths:
    print(f"Processing file: {file_path}")

    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Extract the stock symbol by removing the date pattern from the file name
    df["symbol"] = re.sub(r"_\d{4}_\d{2}\.csv", "", file_path.name)

    # Append the DataFrame to the list of DataFrames
    df_stock_price_list.append(df)

# Concatenate all individual DataFrames into one combined DataFrame
df = pd.concat(df_stock_price_list, ignore_index=True)

# Standardize column names by replacing punctuation
df = replace_punctuation_from_columns(df)

# Remove columns that contain only NA values
df.dropna(how="all", axis=1, inplace=True)

Number of Files Detected: 223
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\BAJAJHFL_2024_09.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\BHAGERIA_2020_05.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\BHAGERIA_2020_06.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\BPCL_2021_08.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\BPCL_2021_09.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\BPCL_2021_10.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\BPCL_2021_11.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\BPCL_2021_12.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\BPCL_2022_01.csv
Processing fi

### Final Processing and Export

- Round numerical values to 2 decimal places.
- Sort the DataFrame by stock name and date.
- Save the processed data as a CSV file in the Silver layer.


In [5]:
# Convert datetime to date string
df["date"] = pd.to_datetime(df["date"]).dt.date

# Round numerical values to 2 decimal places
df = df.round(2)

# Sort the DataFrame by stock name and date
df = df.sort_values(by=["symbol", "date"])

# Select relevant columns
df = df[
    [
        "date",
        "symbol",
        "open",
        "high",
        "low",
        "close",
        "volume",
    ]
]

# Save the result as a CSV file
df.to_csv(stockprice_silver_file_path, index=None)
print("SILVER Layer CSV file for Stock Price history successfully created at:")
print(stockprice_silver_file_path)
# Log the DataFrame debug
df.info()

SILVER Layer CSV file for Stock Price history successfully created at:
C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\SILVER\StockPrice\StockPrice_data.csv
<class 'pandas.core.frame.DataFrame'>
Index: 4330 entries, 0 to 4329
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    4330 non-null   object 
 1   symbol  4330 non-null   object 
 2   open    4330 non-null   float64
 3   high    4330 non-null   float64
 4   low     4330 non-null   float64
 5   close   4330 non-null   float64
 6   volume  4330 non-null   int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 270.6+ KB
